In [ ]:
import os
import torch
import matplotlib.pyplot as plt
from torchvision.models import detection
import numpy as np
import cv2
from skimage.io import imread
from collections import Counter
from numpy import loadtxt

#yaml tqdm

In [ ]:
model_name = 'yolov5' # 'faster' 'mask' 'yolov5' 'yolox'
images_folder = './images/signal_images/'
detected_thresh = 0.4
dim = (1000,300)
CUDA_LAUNCH_BLOCKING=1
COCO_INSTANCE_CATEGORY_NAMES = open("coco_names.txt", "r").read().split(",")
# print(COCO_INSTANCE_CATEGORY_NAMES)

In [ ]:
if model_name == 'faster':
    model = detection.fasterrcnn_resnet50_fpn(pretrained=True)
elif model_name == 'mask':
    model = detection.maskrcnn_resnet50_fpn(pretrained=True)
elif model_name == 'yolov5':
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', force_reload=True, pretrained=True, device='cpu')
    model.conf = detected_thresh
else:
    print("No Model Selected!")

model.eval()

In [ ]:
data_list = os.listdir(images_folder)
pred = []

for i in range(len(data_list)):
    img_path = os.path.join(images_folder, data_list[i])
    # print(img_path)
    image = imread(img_path) 
    if image.size == 0:
        print("No Images in the Folder")
        continue
    image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    if model_name == 'yolov5':
        image1 = image
        predictions = model(image1)
        predictions.save()
        predictions.pandas().xyxy[0]
    else:
        image1 = image.transpose((2, 0, 1))    
        image1 = np.expand_dims(image1, axis=0)
        image1 = image1 / 255.0
        image1 = torch.FloatTensor(image1)

        predictions = model(image1)
        boxes = predictions[0]['boxes'].detach()
        boxes = boxes.cpu().numpy().astype(np.uint16).tolist()
        scores = predictions[0]['scores'].detach().cpu().numpy().tolist()
        labels = predictions[0]['labels'].detach().cpu().numpy().tolist()
        for label, box, s in zip(labels, boxes, scores):
            if s < detected_thresh:
                continue
            pred.append([i,s,[COCO_INSTANCE_CATEGORY_NAMES[label],box]])
            cv2.rectangle(image, (box[0], box[1]), (box[2], box[3] ), color=(0,255,0), thickness=2)
            pred_class = COCO_INSTANCE_CATEGORY_NAMES[label]
            conf = "{} %".format(round(s,2))
            cv2.putText(image,pred_class, (box[0]-10, box[1]-20), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,255,0),thickness=1)
            cv2.putText(image,conf, (box[0]-10, box[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,255,0),thickness=1)
        cv2.imwrite('./results/images_detected_{}_{}.png'.format(model_name, i),image)